In [4]:
import pandas as pd
df = pd.read_excel(
    r"C:\Users\vaiva\OneDrive\Desktop\RD-INFRO-DATA-SCIENCE\Dataset\Telco_customer_churn.xlsx"
)
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [5]:
# Drop identifier
if "CustomerID" in df.columns:
    df = df.drop("CustomerID", axis=1)

# Fix Total Charges column
df["Total Charges"] = pd.to_numeric(df["Total Charges"], errors="coerce")
df["Total Charges"].fillna(df["Total Charges"].mean(), inplace=True)

In [6]:
df = pd.get_dummies(df, drop_first=True)

In [7]:
X = df.drop("Churn Value", axis=1)
y = df["Churn Value"]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print("Model trained successfully")

Model trained successfully


In [10]:
import joblib
joblib.dump(model, "churn_model.pkl")
print("Model saved successfully as churn_model.pkl")

Model saved successfully as churn_model.pkl


In [11]:
# STEP 3: Creating Flask App
from flask import Flask, request
app = Flask(__name__)

In [12]:
# STEP 4: Creating /predict API Endpoint
@app.route('/predict', methods=['POST'])
def predict():
    # Client se aane wala JSON data read kar rahe hain
    data = request.json
    input_data = np.array(list(data.values())).reshape(1, -1)
    # Model se prediction nikal rahe hain
    prediction = model.predict(input_data)
    # Prediction ko readable response me convert kar rahe hain
    result = {
        "prediction": int(prediction[0]),
        "message": "Customer will churn" if prediction[0] == 1 else "Customer will not churn"
    }
    return jsonify(result)

In [ ]:
app = Flask(__name__)
# Load model
model = joblib.load("churn_model.pkl")
@app.route("/")
def home():
    return "Flask API is running successfully!"
@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    if data is None:
        return jsonify({"error": "No input provided"})
    input_df = pd.DataFrame([data])
    prediction = model.predict(input_df)
    return jsonify({
        "prediction": int(prediction[0]),
        "message": "Customer will churn"
        if prediction[0] == 1 else
        "Customer will not churn"
    })
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jan/2026 22:00:55] "GET / HTTP/1.1" 200 -
